In [29]:
import csv
import pandas as pd

def load_csv(csv_filename, columns, to_flatten, value):
    with open(csv_filename, 'r', newline='\n') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        
        data = {}
        header = {elt: index for index, elt in enumerate(next(csv_reader))}
        flattens = set()
        
        for row in csv_reader:
            key = tuple(row[header[column]] for column in columns)
            flattens.add(row[header[to_flatten]])
            try:
                data[key][row[header[to_flatten]]] = row[header[value]]
            except KeyError:
                data[key] = {row[header[to_flatten]]: row[header[value]]}
                
        df_dict = {elt:  [] for elt in columns}
        df_dict.update({elt: [] for elt in flattens})
    
        for key, values in data.items():
            for index, column in enumerate(columns):
                df_dict[column].append(key[index])
            for flatten in flattens:
                df_dict[flatten].append(values.get(flatten, 0)) 
        
        return pd.DataFrame(df_dict)

In [ ]:
p_columns = ['LIBELLE_ARRET', 'CAT_JOUR', 'ID_REFA_LDA']
p_flatten = 'TRNC_HORR_60'
p_value = 'pourc_validations'

v_columns = ['LIBELLE_ARRET', 'JOUR', 'ID_REFA_LDA']
v_flatten = 'CATEGORIE_TITRE'
v_value = 'NB_VALD'

profile_s1 = load_csv('Data/validations-profils-horaires-2017s1.csv', p_columns, p_flatten, p_value)
profile_s2 = load_csv('Data/validations-profils-horaires-2017s2.csv', p_columns, p_flatten, p_value)

validation_s1 = load_csv('Data/validations-nombre-par-jour-2017s1.csv', v_columns, v_flatten, v_value)
validation_s2 = load_csv('Data/validations-nombre-par-jour-2017s2.csv', v_columns, v_flatten, v_value)

In [28]:
validation_s1

,AMETHYSTE,AUTRE TITRE,FGT,ID_REFA_LDA,IMAGINE R,JOUR,LIBELLE_ARRET,NAVIGO,NON DEFINI,TST
0,1310,349,2568,71426,3640,2017-03-17,BARBES-ROCHECHOUART,21335,0,3131
1,518,264,1298,71785,3283,2017-03-17,MAIRIE DE MONTREUIL,16509,0,1457
2,294,94,717,72168,1807,2017-03-17,MAIRIE DE SAINT-OUEN,9145,0,756
3,309,172,606,71909,2332,2017-03-17,MAIRIE DES LILAS,9796,0,542
4,352,134,391,70488,1916,2017-03-17,MAIRIE D'IVRY,5908,0,385
5,224,140,217,70358,1311,2017-03-17,MAISONS-ALFORT-LES JUILLIOTTES,4350,0,287
6,142,88,146,70426,1271,2017-03-17,MAISONS-ALFORT-STADE,3544,0,135
7,158,127,239,73766,2394,2017-03-17,MALAKOFF-PLATEAU DE VANVES,7536,0,227
8,115,29,135,71416,2428,2017-03-17,MALESHERBES,5626,0,162
9,325,161,432,71703,1111,2017-03-17,MARAICHERS,5790,0,423
